# INT 檔案分析交互式測試 / INT File Analysis Interactive Test

**作者 / Author**: Odindino  
**最後更新 / Last Updated**: 2025-06-07  
**版本 / Version**: 1.0

這個筆記本用於測試 KEEN 系統中 .int 檔案的各種分析功能，包括：
- 📂 載入和顯示 .int 檔案
- 🎛️ 交互式選擇平面化方式
- 📍 點選座標提取線段剖面
- 📊 各種統計分析和視覺化

## 使用說明 / Usage Instructions

1. 執行所有 cell 來載入必要的模組和函數
2. 在檔案載入區域選擇要分析的 .int 檔案
3. 使用交互式小工具選擇平面化方式
4. 點擊形貌圖選擇起始和終點來提取線段剖面
5. 查看各種分析結果和統計數據


## 📦 模組導入 / Module Imports

In [1]:
import sys
import os

# 添加 backend 路徑到 Python 路徑
backend_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
if backend_path not in sys.path:
    sys.path.insert(0, backend_path)

import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import json
import warnings
warnings.filterwarnings('ignore')

# 導入 KEEN 模組
from core.experiment_session import ExperimentSession
from core.parsers.int_parser import IntParser
from core.analyzers.int_analyzer import IntAnalyzer
from core.analysis.int_analysis import IntAnalysis
from core.data_models import TopoData

print("✅ 所有模組載入成功 / All modules loaded successfully")

✅ 所有模組載入成功 / All modules loaded successfully


## 🎛️ 交互式控制工具 / Interactive Control Tools

In [2]:
# 全局變數 / Global variables
session = None
current_analyzer = None
current_topo_data = None
original_data = None
selected_points = []
line_profile_data = None

# 建立交互式小工具 / Create interactive widgets
file_path_widget = widgets.Text(
    value='../../../testfile/20250521_Janus Stacking SiO2_13K_113TopoFwd.int',
    placeholder='輸入 .int 檔案路徑 / Enter .int file path',
    description='檔案路徑:',
    style={'description_width': '100px'},
    layout={'width': '600px'}
)

load_button = widgets.Button(
    description='載入檔案 / Load File',
    button_style='primary',
    icon='upload'
)

# 平面化方法選擇
flatten_method_widget = widgets.Dropdown(
    options=[
        ('無平面化 / No Flattening', 'none'),
        ('按行減去均值 / Linewise Mean', 'linewise_mean'),
        ('按行多項式擬合 / Linewise Polyfit', 'linewise_polyfit'),
        ('全局平面擬合 / Plane Flatten', 'plane_flatten'),
        ('2D多項式擬合 / 2D Polynomial', 'polynomial_2d')
    ],
    value='none',
    description='平面化方法:',
    style={'description_width': '100px'}
)

# 多項式階數 (for polyfit methods)
poly_degree_widget = widgets.IntSlider(
    value=1,
    min=1,
    max=5,
    step=1,
    description='多項式階數:',
    style={'description_width': '100px'}
)

apply_flatten_button = widgets.Button(
    description='應用平面化 / Apply Flattening',
    button_style='success',
    icon='magic'
)

reset_button = widgets.Button(
    description='重置到原始數據 / Reset to Original',
    button_style='warning',
    icon='refresh'
)

# 線段剖面方法選擇
profile_method_widget = widgets.Dropdown(
    options=[
        ('插值方法 / Interpolation', 'interpolation'),
        ('Bresenham算法 / Bresenham', 'bresenham')
    ],
    value='interpolation',
    description='剖面方法:',
    style={'description_width': '100px'}
)

clear_points_button = widgets.Button(
    description='清除選點 / Clear Points',
    button_style='danger',
    icon='trash'
)

# 輸出區域
output_widget = widgets.Output()
plot_output = widgets.Output()
profile_output = widgets.Output()
stats_output = widgets.Output()

print("✅ 交互式控制工具創建完成 / Interactive controls created")

✅ 交互式控制工具創建完成 / Interactive controls created


## 🔧 核心功能函數 / Core Functionality Functions

In [3]:
def load_int_file(file_path):
    """載入 .int 檔案 / Load .int file"""
    global session, current_analyzer, current_topo_data, original_data
    
    try:
        with output_widget:
            clear_output()
            print(f"📂 正在載入檔案: {file_path}")
            
        # 檢查檔案是否存在
        if not os.path.exists(file_path):
            with output_widget:
                print(f"❌ 檔案不存在: {file_path}")
            return False
            
        # 使用 IntParser 直接解析
        parser = IntParser()
        parse_result = parser.parse(file_path)
        
        if not parse_result['success']:
            with output_widget:
                print(f"❌ 解析失敗: {parse_result.get('error', '未知錯誤')}")
            return False
            
        # 創建 TopoData
        topo_data = TopoData.from_parse_result(parse_result)
        
        # 創建分析器
        current_analyzer = IntAnalyzer(topo_data)
        current_topo_data = topo_data.image.copy()
        original_data = topo_data.image.copy()
        
        with output_widget:
            print(f"✅ 檔案載入成功!")
            print(f"📊 數據尺寸: {current_topo_data.shape}")
            print(f"📏 掃描範圍: {topo_data.x_range:.1f} × {topo_data.y_range:.1f} nm")
            print(f"🔍 像素尺度: {topo_data.pixel_scale_x:.3f} nm/pixel")
            print(f"📈 高度範圍: {np.min(current_topo_data):.2f} ~ {np.max(current_topo_data):.2f} nm")
            
        # 顯示原始形貌圖
        display_topography()
        display_statistics()
        
        return True
        
    except Exception as e:
        with output_widget:
            print(f"❌ 載入檔案時發生錯誤: {str(e)}")
        return False

def display_topography():
    """顯示形貌圖 / Display topography"""
    global current_topo_data, current_analyzer
    
    if current_topo_data is None or current_analyzer is None:
        return
        
    with plot_output:
        clear_output(wait=True)
        
        # 獲取物理尺寸
        topo_data = current_analyzer.data
        x_range = topo_data.x_range
        y_range = topo_data.y_range
        
        # 創建 Plotly 圖表
        fig = go.Figure(data=go.Heatmap(
            z=current_topo_data,
            x=np.linspace(0, x_range, current_topo_data.shape[1]),
            y=np.linspace(0, y_range, current_topo_data.shape[0]),
            colorscale='viridis',
            colorbar=dict(
                title='Height (nm)',
                titleside='right'
            )
        ))
        
        # 添加選中的點
        if selected_points:
            for i, point in enumerate(selected_points):
                fig.add_trace(go.Scatter(
                    x=[point[1] * topo_data.pixel_scale_x],
                    y=[point[0] * topo_data.pixel_scale_y],
                    mode='markers',
                    marker=dict(size=10, color='red', symbol='x'),
                    name=f'Point {i+1}',
                    showlegend=False
                ))
            
            # 如果有兩個點，添加連線
            if len(selected_points) == 2:
                x_coords = [p[1] * topo_data.pixel_scale_x for p in selected_points]
                y_coords = [p[0] * topo_data.pixel_scale_y for p in selected_points]
                fig.add_trace(go.Scatter(
                    x=x_coords,
                    y=y_coords,
                    mode='lines',
                    line=dict(color='red', width=2, dash='dash'),
                    name='Profile Line',
                    showlegend=False
                ))
        
        fig.update_layout(
            title='SPM 形貌圖 / SPM Topography (點擊選擇線段剖面起終點)',
            xaxis_title='X (nm)',
            yaxis_title='Y (nm)',
            width=700,
            height=600,
            xaxis=dict(scaleanchor="y", constrain='domain')
        )
        
        fig.show()

def display_statistics():
    """顯示統計數據 / Display statistics"""
    global current_topo_data, current_analyzer
    
    if current_topo_data is None:
        return
        
    with stats_output:
        clear_output(wait=True)
        
        # 計算統計數據
        stats = IntAnalysis.get_topo_stats(current_topo_data)
        roughness = IntAnalysis.calculate_surface_roughness(current_topo_data)
        
        print("📊 數據統計 / Data Statistics")
        print("=" * 40)
        print(f"最小值 / Min:     {stats['min']:.3f} nm")
        print(f"最大值 / Max:     {stats['max']:.3f} nm")
        print(f"平均值 / Mean:    {stats['mean']:.3f} nm")
        print(f"中位數 / Median:  {stats['median']:.3f} nm")
        print(f"標準差 / Std:     {stats['std']:.3f} nm")
        print(f"RMS:              {stats['rms']:.3f} nm")
        print()
        print("🔍 表面粗糙度 / Surface Roughness")
        print("=" * 40)
        print(f"Ra (算術平均):    {roughness['Ra']:.3f} nm")
        print(f"Rq (均方根):      {roughness['Rq']:.3f} nm")
        print(f"Rz (峰谷值):      {roughness['Rz']:.3f} nm")
        print(f"Rp (最大峰高):    {roughness['Rp']:.3f} nm")
        print(f"Rv (最大谷深):    {roughness['Rv']:.3f} nm")

def apply_flattening(method, degree=1):
    """應用平面化 / Apply flattening"""
    global current_analyzer, current_topo_data
    
    if current_analyzer is None:
        with output_widget:
            print("❌ 請先載入 .int 檔案")
        return
        
    try:
        with output_widget:
            clear_output()
            print(f"🔄 正在應用平面化方法: {method}")
            
        if method == 'none':
            # 重置到原始數據
            current_analyzer.reset_to_original()
            current_topo_data = current_analyzer.current_topo_data.copy()
            with output_widget:
                print("✅ 已重置到原始數據")
        else:
            # 應用選定的平面化方法
            if method == 'polynomial_2d':
                result = current_analyzer.apply_flattening(method, order=degree)
            elif method == 'linewise_polyfit':
                result = current_analyzer.apply_flattening(method, degree=degree)
            else:
                result = current_analyzer.apply_flattening(method)
                
            if result['success']:
                current_topo_data = result['data']['flattened_data']
                with output_widget:
                    print(f"✅ 平面化完成: {method}")
            else:
                with output_widget:
                    print(f"❌ 平面化失敗: {result.get('error', '未知錯誤')}")
                return
                
        # 更新顯示
        display_topography()
        display_statistics()
        
    except Exception as e:
        with output_widget:
            print(f"❌ 平面化過程中發生錯誤: {str(e)}")

def on_plot_click(x, y):
    """處理圖表點擊事件 / Handle plot click events"""
    global selected_points, current_analyzer
    
    if current_analyzer is None:
        return
        
    # 轉換物理坐標到像素坐標
    topo_data = current_analyzer.data
    pixel_x = int(x / topo_data.pixel_scale_x)
    pixel_y = int(y / topo_data.pixel_scale_y)
    
    # 確保坐標在範圍內
    pixel_x = max(0, min(pixel_x, current_topo_data.shape[1] - 1))
    pixel_y = max(0, min(pixel_y, current_topo_data.shape[0] - 1))
    
    selected_points.append((pixel_y, pixel_x))
    
    with output_widget:
        print(f"📍 選中點 {len(selected_points)}: ({pixel_x}, {pixel_y}) = ({x:.1f}, {y:.1f}) nm")
        
    # 如果選擇了兩個點，自動提取線段剖面
    if len(selected_points) == 2:
        extract_line_profile()
    elif len(selected_points) > 2:
        # 重置選點
        selected_points = [selected_points[-1]]  # 保留最後一個點
        with output_widget:
            print("🔄 重置選點，開始新的線段選擇")
            
    # 更新顯示
    display_topography()

def extract_line_profile():
    """提取線段剖面 / Extract line profile"""
    global selected_points, current_analyzer, line_profile_data
    
    if len(selected_points) != 2 or current_analyzer is None:
        with output_widget:
            print("❌ 請選擇兩個點來定義線段")
        return
        
    try:
        method = profile_method_widget.value
        start_point = selected_points[0]
        end_point = selected_points[1]
        
        with output_widget:
            print(f"📏 正在提取線段剖面 ({method})...")
            
        # 提取線段剖面
        result = current_analyzer.extract_line_profile(start_point, end_point, method)
        
        if result['success']:
            line_profile_data = result['data']
            
            with output_widget:
                print(f"✅ 線段剖面提取成功")
                print(f"📊 剖面長度: {line_profile_data['length']:.2f} nm")
                print(f"🔍 採樣點數: {len(line_profile_data['height'])}")
                
            # 顯示剖面圖
            display_line_profile()
            
        else:
            with output_widget:
                print(f"❌ 線段剖面提取失敗: {result.get('error', '未知錯誤')}")
                
    except Exception as e:
        with output_widget:
            print(f"❌ 提取線段剖面時發生錯誤: {str(e)}")

def display_line_profile():
    """顯示線段剖面圖 / Display line profile"""
    global line_profile_data
    
    if line_profile_data is None:
        return
        
    with profile_output:
        clear_output(wait=True)
        
        # 創建線段剖面圖
        fig = go.Figure()
        
        fig.add_trace(go.Scatter(
            x=line_profile_data['distance'],
            y=line_profile_data['height'],
            mode='lines',
            name='Height Profile',
            line=dict(color='blue', width=2)
        ))
        
        fig.update_layout(
            title='線段高度剖面 / Line Height Profile',
            xaxis_title='Distance (nm)',
            yaxis_title='Height (nm)',
            width=700,
            height=400,
            showlegend=False
        )
        
        fig.show()
        
        # 顯示剖面統計
        if 'stats' in line_profile_data:
            stats = line_profile_data['stats']
            print("📊 剖面統計 / Profile Statistics")
            print("=" * 35)
            print(f"長度 / Length:    {line_profile_data['length']:.2f} nm")
            print(f"最小值 / Min:     {stats['min']:.3f} nm")
            print(f"最大值 / Max:     {stats['max']:.3f} nm")
            print(f"平均值 / Mean:    {stats['mean']:.3f} nm")
            print(f"範圍 / Range:     {stats['range']:.3f} nm")
            print(f"標準差 / Std:     {stats['std']:.3f} nm")
            print(f"RMS:              {stats['rms']:.3f} nm")

print("✅ 核心功能函數創建完成 / Core functions created")

✅ 核心功能函數創建完成 / Core functions created


## 🔗 事件處理函數 / Event Handler Functions

In [4]:
# 事件處理函數 / Event handlers
def on_load_button_click(b):
    """載入按鈕點擊事件 / Load button click event"""
    file_path = file_path_widget.value.strip()
    if file_path:
        load_int_file(file_path)
    else:
        with output_widget:
            print("❌ 請輸入檔案路徑")

def on_apply_flatten_click(b):
    """應用平面化按鈕點擊事件 / Apply flattening button click event"""
    method = flatten_method_widget.value
    degree = poly_degree_widget.value
    apply_flattening(method, degree)

def on_reset_button_click(b):
    """重置按鈕點擊事件 / Reset button click event"""
    apply_flattening('none')

def on_clear_points_click(b):
    """清除選點按鈕點擊事件 / Clear points button click event"""
    global selected_points, line_profile_data
    selected_points = []
    line_profile_data = None
    
    with output_widget:
        print("🧹 已清除所有選點")
    
    with profile_output:
        clear_output()
        
    display_topography()

def on_method_change(change):
    """平面化方法改變事件 / Flattening method change event"""
    # 根據選擇的方法顯示/隱藏多項式階數設置
    if change['new'] in ['linewise_polyfit', 'polynomial_2d']:
        poly_degree_widget.layout.visibility = 'visible'
    else:
        poly_degree_widget.layout.visibility = 'hidden'

# 綁定事件 / Bind events
load_button.on_click(on_load_button_click)
apply_flatten_button.on_click(on_apply_flatten_click)
reset_button.on_click(on_reset_button_click)
clear_points_button.on_click(on_clear_points_click)
flatten_method_widget.observe(on_method_change, names='value')

print("✅ 事件處理函數綁定完成 / Event handlers bound")

✅ 事件處理函數綁定完成 / Event handlers bound


## 🎮 交互式控制界面 / Interactive Control Interface

In [5]:
# 創建控制界面 / Create control interface
print("🎮 INT 檔案分析交互式控制界面 / INT File Analysis Interactive Control Interface")
print("=" * 80)

# 檔案載入區域
print("\n📂 檔案載入 / File Loading")
display(widgets.HBox([file_path_widget, load_button]))

# 平面化控制區域
print("\n🎛️ 平面化控制 / Flattening Control")
control_box = widgets.VBox([
    widgets.HBox([flatten_method_widget, poly_degree_widget]),
    widgets.HBox([apply_flatten_button, reset_button])
])
display(control_box)

# 線段剖面控制區域
print("\n📏 線段剖面控制 / Line Profile Control")
profile_control_box = widgets.HBox([
    profile_method_widget, 
    clear_points_button
])
display(profile_control_box)

print("\n📊 使用說明 / Instructions:")
print("1. 在檔案路徑欄輸入 .int 檔案的路徑，然後點擊 '載入檔案'")
print("2. 選擇平面化方法，然後點擊 '應用平面化'")
print("3. 在形貌圖上點擊兩個點來選擇線段剖面的起點和終點")
print("4. 系統會自動提取並顯示線段剖面圖")
print("5. 使用 '清除選點' 來重新選擇，'重置到原始數據' 來取消平面化")

print("\n📝 輸出區域 / Output Areas:")

🎮 INT 檔案分析交互式控制界面 / INT File Analysis Interactive Control Interface

📂 檔案載入 / File Loading



🎛️ 平面化控制 / Flattening Control



📏 線段剖面控制 / Line Profile Control



📊 使用說明 / Instructions:
1. 在檔案路徑欄輸入 .int 檔案的路徑，然後點擊 '載入檔案'
2. 選擇平面化方法，然後點擊 '應用平面化'
3. 在形貌圖上點擊兩個點來選擇線段剖面的起點和終點
4. 系統會自動提取並顯示線段剖面圖
5. 使用 '清除選點' 來重新選擇，'重置到原始數據' 來取消平面化

📝 輸出區域 / Output Areas:


## 📤 狀態輸出 / Status Output

In [ ]:
print("📤 系統狀態 / System Status")
display(output_widget)

## 🗺️ 形貌圖顯示 / Topography Display

In [ ]:
print("🗺️ 形貌圖 / Topography Map")
print("點擊圖上的兩個點來選擇線段剖面 / Click two points to select line profile")
display(plot_output)

## 📊 統計數據顯示 / Statistics Display

In [ ]:
print("📊 數據統計 / Data Statistics")
display(stats_output)

## 📈 線段剖面顯示 / Line Profile Display

In [ ]:
print("📈 線段剖面 / Line Profile")
display(profile_output)

## 🧪 高級測試功能 / Advanced Testing Features

In [ ]:
# 手動測試區域 / Manual testing area
print("🧪 高級測試功能 / Advanced Testing Features")
print("=" * 50)
print("在這個 cell 中，您可以手動執行各種測試：")
print()
print("# 範例：直接使用分析器功能")
print("# if current_analyzer is not None:")
print("#     # 測試傾斜校正")
print("#     result = current_analyzer.apply_tilt_correction('up', step_size=5)")
print("#     print(result)")
print()
print("#     # 測試特徵檢測")
print("#     features = current_analyzer.detect_features('peaks')")
print("#     print(f'檢測到 {features[\"data\"][\"count\"]} 個峰值')")
print()
print("# 您可以取消註釋並修改上面的代碼來測試不同功能")

# 可以在這裡添加自定義測試代碼
# Add custom test code here


## 📋 功能測試清單 / Feature Testing Checklist

### ✅ 已實現功能 / Implemented Features

- [x] **檔案載入 / File Loading**
  - [x] .int 檔案解析
  - [x] 數據模型轉換
  - [x] 錯誤處理

- [x] **平面化處理 / Flattening Processing**
  - [x] 按行減去均值 (Linewise Mean)
  - [x] 按行多項式擬合 (Linewise Polyfit)
  - [x] 全局平面擬合 (Plane Flatten)
  - [x] 2D多項式擬合 (2D Polynomial)
  - [x] 重置到原始數據

- [x] **線段剖面分析 / Line Profile Analysis**
  - [x] 插值方法 (Interpolation)
  - [x] Bresenham 算法
  - [x] 交互式點選
  - [x] 剖面統計

- [x] **視覺化 / Visualization**
  - [x] 交互式形貌圖
  - [x] 線段剖面圖
  - [x] 統計數據顯示

### 🔄 可擴展功能 / Expandable Features

- [ ] **傾斜校正 / Tilt Correction**
- [ ] **表面特徵檢測 / Surface Feature Detection**
- [ ] **FFT 分析 / FFT Analysis**
- [ ] **批次處理 / Batch Processing**
- [ ] **數據導出 / Data Export**

### 📝 測試建議 / Testing Recommendations

1. **基礎測試 / Basic Testing**:
   - 載入不同的 .int 檔案
   - 嘗試各種平面化方法
   - 在不同位置提取線段剖面

2. **邊界測試 / Boundary Testing**:
   - 測試檔案不存在的情況
   - 測試無效的檔案路徑
   - 測試極端的座標選擇

3. **性能測試 / Performance Testing**:
   - 大尺寸圖像的處理速度
   - 複雜平面化方法的計算時間
   - 記憶體使用情況

4. **準確性驗證 / Accuracy Verification**:
   - 與已知標準進行比較
   - 檢查統計數據的正確性
   - 驗證線段剖面的準確性